In [2]:
!pip install transformers
!pip install datasets
!pip install evaluate

#from consts import ProductType, Sentiment
#from amazon_scrapping import get_scrapped_reviews, filter_and_format_reviews

import transformers
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from transformers import AutoModelForSequenceClassification
import pandas as pd
from typing import Sequence, Dict
from sklearn.model_selection import train_test_split
import numpy as np
import evaluate
import pickle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 28.2 MB/s 
     |████████████████████████████████| 212 kB 48.1 MB/s 
     |████████████████████████████████| 132 kB 60.2 MB/s 
     |████████████████████████████████| 127 kB 60.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 1.4 MB/s 


In [3]:
label_to_numeric: Dict[str, int] = {"POSITIVE": 0, "NEUTRAL": 1, "NEGATIVE": 2}

In [ ]:
reviews_with_labels: list[tuple[str, Sentiment]] = []

for product in ProductType:
    raw_reviews = get_scrapped_reviews(product_type=product, inout_folder="scrapped_data")
    reviews_with_labels.extend(filter_and_format_reviews(raw_reviews=raw_reviews, suppress_errors=True))

reviews: Sequence[str] = [rev_label[0] for rev_label in reviews_with_labels]
labels: Sequence[int] = [label_to_numeric[rev_label[1]] for rev_label in reviews_with_labels]

In [ ]:
df_text: pd.DataFrame = pd.DataFrame({"text": reviews})
df_labels: pd.DataFrame = pd.DataFrame({"label": labels})

X_train, X_test, Y_train, Y_test = train_test_split(df_text, df_labels, test_size=0.25, random_state=40)


df_train: pd.DataFrame = pd.concat([X_train, Y_train], axis=1)
df_test: pd.DataFrame = pd.concat([X_test, Y_test], axis=1)

print(df_train["label"].value_counts())
print(df_test["label"].value_counts())

df_train.to_csv("./csv_data/train.csv", index=False)
df_test.to_csv("./csv_data/test.csv", index=False)
df_train.head()

0    4691
2    1033
1     609
Name: label, dtype: int64
0    1555
2     375
1     181
Name: label, dtype: int64


,text,label
5549,Diese Tabs haben eine neuartige Machart mit we...,0
1038,Ich bin total begeistert von der Emsa Travel M...,0
2126,Ich kann es nur empfehlen. Hält sehr lange war...,0
5204,Ich habe die Edelstahlausführung zu Weihnachte...,2
3567,Einer der Deckel klemmt ein bischen und müsste...,2


In [6]:
df_train = pd.read_csv("./csv_data/train.csv")
df_test = pd.read_csv("./csv_data/test.csv")

dataset_train: Dataset = Dataset.from_pandas(df_train)
dataset_test: Dataset = Dataset.from_pandas(df_test)

In [7]:
tokenizer: transformers.models.bert.tokenization_bert_fast.BertTokenizerFast = AutoTokenizer.from_pretrained("bert-base-german-cased")

def tokenize_function(data: Dataset) -> Dataset:
    return tokenizer(data["text"], padding="max_length", truncation=True)

train_dataset_tokenized: Dataset = dataset_train.map(tokenize_function, batched=True)
test_dataset_tokenized: Dataset = dataset_test.map(tokenize_function, batched=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485k [00:00<?, ?B/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-german-cased", num_labels=3)

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoi

In [9]:
metric: evaluate.EvaluationModule = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args: TrainingArguments = TrainingArguments(output_dir="checkpoints", evaluation_strategy="epoch", num_train_epochs=3, per_device_train_batch_size=16)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 6333
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1188
  Number of trainable parameters = 109083651
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.716291,0.751776
2,0.760200,0.732533,0.751776
3,0.679200,0.771377,0.727144


***** Running Evaluation *****
  Num examples = 2111
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to checkpoints/checkpoint-500
Configuration saved in checkpoints/checkpoint-500/config.json
Model weights saved in checkpoints/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2111
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to checkpoints/checkpoint-1000
Configuration saved in checkpoints/checkpoint-1000/config.json
Model weights saved in checkpoints/checkpo

TrainOutput(global_step=1188, training_loss=0.699594998600507, metrics={'train_runtime': 1945.4815, 'train_samples_per_second': 9.766, 'train_steps_per_second': 0.611, 'total_flos': 4998891823414272.0, 'train_loss': 0.699594998600507, 'epoch': 3.0})

In [11]:
predictions = trainer.predict(test_dataset_tokenized)
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 2111
  Batch size = 8
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


(2111, 3) (2111,)


In [15]:
from datasets import load_metric

metric = load_metric("accuracy")
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.7271435338702037}

In [26]:
idx = 2
print(dataset_test['text'][idx])
print(dataset_test['label'][idx])

Nicht gut
0
